In [5]:
# Load LSTM network and generate text
import sys
import numpy
from numpy import array
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense, Dropout , BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from keras import optimizers
from keras.layers import Reshape
from keras.models import load_model
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import optimizers

# load training data label in text form
raw_train_label_text = open('train_label.txt', 'r', encoding='utf-8').read()
raw_test_label_text =  open('test_label.txt', 'r', encoding='utf-8').read()

# convert text to integer
test_x = []
train_x = []
for _ in raw_train_label_text.splitlines():
    val = int(_)
    train_x.append(val)
for _ in raw_test_label_text.splitlines():
    val = int(_)
    test_x.append(val)

# summarize the loaded data
train_count = len(train_x)
test_count = len(test_x)
print ("Total Train: ", train_count)
print ("Total Test: ", test_count)

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 1
n_neurons = 10
# split into samples
TrainX, trainy = split_sequence(train_x, n_steps)
TestX, testy = split_sequence(test_x, n_steps)

print(TrainX.shape, trainy.shape)
print(TestX.shape, testy.shape)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
n_neurons = 50
TrainX = TrainX.reshape((TrainX.shape[0], n_features, n_features))
TestX = TestX.reshape((TestX.shape[0], n_features, n_features))

print(TrainX.shape, trainy.shape)
print(TestX.shape, testy.shape)
n_batch = 1

# define model
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, TrainX.shape[1], TrainX.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
 
# fit model
#model.fit(TrainX, trainy, epochs=10, verbose=1, shuffle=False)
#_, accuracy = model.evaluate(TestX, testy, batch_size=10, verbose=1)
#print("Accuracy: {:.2f} %".format(accuracy*100))
# demonstrate prediction
# fit network
for i in range(10):
	model.fit(TrainX, trainy, epochs=10, batch_size=n_batch, verbose=0, shuffle=False)
	model.reset_states()
# online forecast
for i in range(len(TestX)):
    x, y = TestX[i], testy[i]
    x = x.reshape(1, 1, 1)
    yhat = model.predict(x, batch_size=1)
    print('>Expected=%.1f, Predicted=%.1f' % (y, yhat))


Total Train:  11084
Total Test:  5225
(11083, 1) (11083,)
(5224, 1) (5224,)
(11083, 1, 1) (11083,)
(5224, 1, 1) (5224,)


KeyboardInterrupt: 